In [ ]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [ ]:
# Import modules
import numpy as np # maths
from scipy.interpolate import interp1d
from scipy.optimize import root_scalar

from time import time# timer for debug

    # gather results obtained from the runs
import pickle # open .pkl files where python objects have been saved
from safe import safe # the empty class which is used to save the results
    
    # system commands
import os
    
    # interact with the C++ core and the Python core
import dimers as dim # C++ interface module
import KagomeFunctions as kf # "library" allowing to work on Kagome
import KagomeDrawing as kdraw # "library" allowing to plot Kagome
import Observables as obs #observables that can be used by the run script
import KagomeFT as kft #small functions to compute the FT
import AnalysisFunctions as af # functions to make the analysis code cleaner
    #plots using matplotlib
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rcParams.update({'font.size': 14, 'pgf.texsystem':'pdflatex'})

In [ ]:
# Create a folder to save the pictures
foldername = '../../Analysis_PSI_EPFL/Runs_14-08-20_TipParameters/'
results_foldername = 'Htip1_Path5_19-08-20'
filenamelist = ['TipParams_L8_htip1_ttip0_path5_pswitch_0.05_uponlyTrue_folder/backup',
                'TipParams_L8_htip1_ttip0_path5_pswitch_0.1_uponlyTrue_folder/backup',
                'TipParams_L8_htip1_ttip0_path5_pswitch_0.15_uponlyTrue_folder/backup',
                'TipParams_L8_htip1_ttip0_path5_pswitch_0.2_uponlyTrue_folder/backup',
                'TipParams_L8_htip1_ttip0_path5_pswitch_0.25_uponlyTrue_folder/backup',
                'TipParams_L8_htip1_ttip0_path5_pswitch_0.3_uponlyTrue_folder/backup',
                'TipParams_L8_htip1_ttip0_path5_pswitch_0.35_uponlyTrue_folder/backup',
                'TipParams_L8_htip1_ttip0_path5_pswitch_0.4_uponlyTrue_folder/backup',
                'TipParams_L8_htip1_ttip0_path5_pswitch_0.45_uponlyTrue_folder/backup',
                'TipParams_L8_htip1_ttip0_path5_pswitch_0.5_uponlyTrue_folder/backup',
                'TipParams_L8_htip1_ttip0_path5_pswitch_0.55_uponlyTrue_folder/backup',
                'TipParams_L8_htip1_ttip0_path5_pswitch_0.6_uponlyTrue_folder/backup',
                'TipParams_L8_htip1_ttip0_path5_pswitch_0.65_uponlyTrue_folder/backup',
                'TipParams_L8_htip1_ttip0_path5_pswitch_0.7_uponlyTrue_folder/backup',
                'TipParams_L8_htip1_ttip0_path5_pswitch_0.75_uponlyTrue_folder/backup',
                'TipParams_L8_htip1_ttip0_path5_pswitch_0.8_uponlyTrue_folder/backup',
                'TipParams_L8_htip1_ttip0_path5_pswitch_0.85_uponlyTrue_folder/backup',
                'TipParams_L8_htip1_ttip0_path5_pswitch_0.9_uponlyTrue_folder/backup',
                'TipParams_L8_htip1_ttip0_path5_pswitch_0.95_uponlyTrue_folder/backup',
                'TipParams_L8_htip1_ttip0_path5_pswitch_1_uponlyTrue_folder/backup'
               ]
os.makedirs('./' + foldername + results_foldername, exist_ok = True)

n = len(filenamelist)

In [ ]:
[L, numsites, J1, J2, J3, J3st, J4, nb, num_in_bin, 
 htip, Ttip, pswitch, uponly, path,
 temperatures, nt,
 stat_temps, temperatures_plots, hfields, nh, 
 stat_hfields, hfields_plots, listfunctions, sref, ids2walker] = \
af.LoadParameters(foldername, filenamelist)
print(listfunctions)
print("Ttip", Ttip)
print("htip", htip)
print("Uponly", uponly)
print("pswitch", pswitch)
print("path", path)
s_ijl, ijl_s = kdraw.createspinsitetable(L[0])
print("nb ", nb, "num_in_bin ", num_in_bin)
swapst_th, swapsh_th, swapst, swapsh = af.LoadSwaps(foldername, filenamelist, nb, num_in_bin, nh, nt)

n = len(L)
failedth, failedssfth, failed, failedssf = \
af.LoadUpdates(foldername, filenamelist, nb, num_in_bin, [9*L[i]**2 for i in range(n)])


kw = {'binning': True, 'plzplot': False, 'plotmin': 0, 'plotmax': 16}
[t_h_MeanE, t_h_MeanEsq, t_h_varMeanE, t_h_varMeanEsq, C, ErrC] = \
 af.LoadEnergy(foldername, filenamelist, numsites,
               nb, stat_temps, temperatures, stat_hfields, listfunctions, **kw)

[t_h_MeanM, t_h_MeanMsq, t_h_varMeanM, t_h_varMeanMsq, Chi, ErrChi] = \
 af.LoadMagnetisation(foldername, filenamelist, numsites,
               nb, stat_temps, temperatures, stat_hfields, listfunctions, **kw)

In [ ]:
###########################################
        ## ALGORITHM ANALYSIS ##
###########################################
tidmin = 0
tidmax = nt
af.SwapsAnalysis(L, n, tidmin, tidmax, temperatures, hfields,
                 foldername, results_foldername, swapst, swapsh)

In [ ]:
af.FailedAnalysis(L, n, tidmin, tidmax, temperatures, hfields,
                 foldername, results_foldername,failed, failedssf)

In [ ]:
tidmin = 0
tidmax = [len(stat_temps[i]) for i in range(n)]
temperatures_plots = np.array(temperatures_plots)

In [ ]:
###########################################
        ## Energy ANALYSIS ##
###########################################
### Energy
#S0 = np.log(2)
#
#for i in range(n):
#    addsave = "ht={0}_Ttip={1}_uponly={2}_pswitch={3}_path={4}_L={5}".format(
#        htip[i],Ttip[i], uponly[i], pswitch[i], path[i], L[i]);
#    kw = {'gscheck': True, 'addsave': addsave}
#    af.BasicPlotsE(L, i, tidmin, tidmax, temperatures_plots, hfields_plots, foldername,
#                results_foldername, filenamelist, t_h_MeanE, t_h_MeanEsq, t_h_varMeanE,
#                t_h_varMeanEsq, C, ErrC, J1, J2, J3, J4,  **kw)
#plt.show()

In [ ]:
###########################################
        ## MAGNETISATION ANALYSIS ##
###########################################
#
#for i in range(n):
#    addsave = "ht={0}_Ttip={1}_uponly={2}_pswitch={3}_path={4}_L={5}".format(
#        htip[i],Ttip[i], uponly[i], pswitch[i], path[i], L[i]);
#    kw = {'expm': 0.193, 'expmerr': 0.01, 'addsave': addsave}
#    af.BasicPlotsM(L, i, tidmin, tidmax, temperatures_plots, hfields_plots, foldername,
#                    results_foldername, filenamelist, t_h_MeanM, t_h_MeanMsq, t_h_varMeanM,
#                    t_h_varMeanMsq, Chi, ErrChi, J1, J2, J3, J4, **kw)
#plt.show()

In [ ]:
###########################################
    ## Correlations load and analysis ##
###########################################

In [ ]:
rmmag = True
kw = {'rmmag':rmmag}

In [ ]:
t_h_MeanFc, t_h_varMeanFc, t_h_MeanSi, t_h_varMeanSi= \
 af.LoadFirstCorrelations(foldername, filenamelist, listfunctions,stat_temps, stat_hfields, nb, t_h_varMeanMsq, **kw)

In [ ]:
#t_h_MeanSs, t_h_varMeanSs, t_h_MeanSi, t_h_varMeanSi, t_h_MeanCorr, t_h_errCorrEstim = \
# af.LoadCentralCorrelations(foldername, filenamelist, listfunctions, sref, stat_temps, stat_hfields, nb, **kw)
#print(t_h_errCorrEstim[0].shape)
#print(t_h_MeanFc[0].shape)

In [ ]:
#for i in range(n):
#    addsave = "ht={0}_Ttip={1}_uponly={2}_pswitch={3}_path={4}_L={5}".format(
#        htip[i],Ttip[i], uponly[i], pswitch[i], path[i], L[i]);
#    addtitle = r"$T_{t}/J_1=$"+"{0},".format(Ttip[i])+r" $h_{t}/J_1$=" +"{0}, $L$ = {1}".format(htip[i], L[i]);
#
#        
#    af.BasicPlotsFirstCorrelations(L, i, t_h_MeanFc, temperatures_plots, 
#                                   t_h_varMeanFc, 
#                                   foldername, results_foldername, 
#                                   filenamelist, tmin = 0, 
#                                   setyticks = None,
#                                   addtitle = addtitle, addsave = addsave,
#                                   save = True, log = True,
#                                   figsize=(9,6), dpi = 200)

In [ ]:
#for i in range(n):
#    addsave = "ht={0}_Ttip={1}_uponly={2}_pswitch={3}_path={4}_L={5}".format(
#        htip[i],Ttip[i], uponly[i], pswitch[i], path[i], L[i]);
#    addtitle = r"$T_{t}/J_1=$"+"{0}".format(Ttip[i])+r", $h_{t}/J_1$=" +"{0}, $L$ = {1}".format(htip[i], L[i]);
#
#        
#    af.BasicPlotsFirstCorrelations(L, i, t_h_MeanFc, temperatures_plots, 
#                                   t_h_varMeanFc, 
#                                   foldername, results_foldername, 
#                                   filenamelist, tmin = 0, 
#                                   setyticks = None,
#                                   addtitle = addtitle, addsave = addsave,
#                                   save = True, log = False,
#                                   figsize=(9,6), dpi = 200)

In [ ]:
# Introducing the experimental values for the plots below:
## <sisj>-<si><sj>:

NN1exp = -0.22;
NN1experr = 0.005;

NN2exp = 0.021;
NN2experr = 0.004;

NN3pexp = 0.064;
NN3pexperr = 0.005;

NN3sexp = -0.001;
NN3sexperr = 0.005;
mexp = -0.193;
mexperr = 0.01;
print(mexp)
rmexpmag = True

if not rmexpmag:
    NN1exp += mexp**2;
    NN2exp += mexp**2;
    NN3pexp += mexp**2;
    NN3sexp += mexp**2;
    

print(NN1exp)
print(NN2exp)
print(NN3pexp)
print(NN3sexp)

In [ ]:
for i in range(n):
    addsave = "ht={0}_Ttip={1}_uponly={2}_pswitch={3}_path={4}_L={5}".format(
        htip[i],Ttip[i], uponly[i], pswitch[i], path[i], L[i]);
    addtitle = r"$T_{t}/J_1=$"+"{0}".format(Ttip[i])+r", $h_{t}/J_1$=" +"{0}, $L$ = {1}".format(htip[i], L[i]) +r", $p$ = {0}".format(pswitch[i]);

        
    af.BasicPlotsFirstCorrelations(L, i, t_h_MeanFc, temperatures_plots, 
                                   t_h_varMeanFc, 
                                   foldername, results_foldername, 
                                   filenamelist, tmin = 0, 
                                   setyticks = None,
                                   addtitle = addtitle, addsave = addsave,
                                   save = True, log = True,
                                   figsize=(9,6), dpi = 200)
    plt.gca().set_prop_cycle(None)
    plt.fill_between([1e-3,60],[NN1exp-NN1experr,NN1exp-NN1experr],[NN1exp+NN1experr, NN1exp+NN1experr], alpha = 0.2, label = r'$c_1$ - exp')
    plt.fill_between([1e-3,60],[NN2exp-NN2experr,NN2exp-NN2experr],[NN2exp+NN2experr, NN2exp+NN2experr], alpha = 0.2, label = r'$c_2$ - exp')
    plt.fill_between([1e-3,60],[NN3pexp-NN3pexperr,NN3pexp-NN3pexperr],[NN3pexp+NN3pexperr, NN3pexp+NN3pexperr], alpha = 0.2, label = r'$c_{3||}$ - exp')
    plt.fill_between([1e-3,60],[NN3sexp-NN3sexperr, NN3sexp-NN3sexperr],[NN3sexp+NN3sexperr, NN3sexp+NN3sexperr], alpha = 0.2, label = r'$c_{3\star}$- exp')
    plt.xlim([0.1, 60])
    plt.ylim([-0.35, 0.2])
    plt.yticks(np.arange(-0.35,0.25,0.05))
    plt.legend()
    plt.savefig("./" + foldername + results_foldername + "/FirstCorrelations_Zoom" + addsave +"rmexpmag={0}.png".format(rmexpmag))

In [ ]:
for i in range(n):
    addsave = "ht={0}_Ttip={1}_uponly={2}_pswitch={3}_path={4}_L={5}".format(
        htip[i],Ttip[i], uponly[i], pswitch[i], path[i], L[i]);
    addtitle = r"$T_{t}/J_1=$"+"{0}".format(Ttip[i])+r", $h_{t}/J_1$=" +"{0}, $L$ = {1}".format(htip[i], L[i]) +r", $p$ = {0}".format(pswitch[i]);

        
    af.BasicPlotsFirstCorrelations(L, i, t_h_MeanFc, temperatures_plots, 
                                   t_h_varMeanFc, 
                                   foldername, results_foldername, 
                                   filenamelist, tmin = 0, 
                                   setyticks = None,
                                   addtitle = addtitle, addsave = addsave,
                                   save = True, log = False,
                                   figsize=(9,6), dpi = 200)
    plt.gca().set_prop_cycle(None)
    plt.fill_between([1e-3,60],[NN1exp-NN1experr,NN1exp-NN1experr],[NN1exp+NN1experr, NN1exp+NN1experr], alpha = 0.2, label = r'$c_1$ - exp')
    plt.fill_between([1e-3,60],[NN2exp-NN2experr,NN2exp-NN2experr],[NN2exp+NN2experr, NN2exp+NN2experr], alpha = 0.2, label = r'$c_2$ - exp')
    plt.fill_between([1e-3,60],[NN3pexp-NN3pexperr,NN3pexp-NN3pexperr],[NN3pexp+NN3pexperr, NN3pexp+NN3pexperr], alpha = 0.2, label = r'$c_{3||}$ - exp')
    plt.fill_between([1e-3,60],[NN3sexp-NN3sexperr, NN3sexp-NN3sexperr],[NN3sexp+NN3sexperr, NN3sexp+NN3sexperr], alpha = 0.2, label = r'$c_{3\star}$- exp')
    plt.xlim([0.1, 60])
    plt.ylim([-0.35, 0.2])
    plt.yticks(np.arange(-0.35,0.25,0.05))
    plt.legend()
    plt.savefig("./" + foldername + results_foldername + "/FirstCorrelations_ZoomLinear" + addsave +"rmexpmag={0}.png".format(rmexpmag))

In [ ]:
ploth = False

In [ ]:
#for i in range(n):
#    addsave = "ht={0}_Ttip={1}_uponly={2}_pswitch={3}_path={4}_L={5}".format(
#        htip[i],Ttip[i], uponly[i], pswitch[i], path[i], L[i]);
#    af.PlotFirstCorrelations(i, L,foldername, results_foldername,  hfields_plots, temperatures_plots,
#                             t_h_MeanCorr, t_h_errCorrEstim, sref, \
#                             distmax = 2, ploth = ploth, plotFirst = False,
#                            t_h_MeanFc = t_h_MeanFc, **kw)
#    plt.title(r"$T_{t}=$"+"{0}".format(Ttip[i])+ ", "+ r"$\frac{h_{t}}{J_1}=$"+"{0}".format(htip[i]))
#    plt.savefig("./" + foldername + results_foldername + "/FirstCorrelations_Central_"+ addsave +".png".format(i))

In [ ]:
#for i in range(n):
#    addsave = "ht={0}_Ttip={1}_uponly={2}_pswitch={3}_path={4}_L={5}".format(
#        htip[i],Ttip[i], uponly[i], pswitch[i], path[i], L[i]);
#    af.PlotFirstCorrelations(i, L,foldername, results_foldername,  hfields_plots, temperatures_plots,
#                             t_h_MeanCorr, t_h_errCorrEstim, sref, \
#                             distmax = 2, ploth = ploth, plotFirst = True,
#                            t_h_MeanFc = t_h_MeanFc, **kw)
#    plt.title(r"$T_{t}=$"+"{0}".format(Ttip[i])+ ", "+ r"$\frac{h_{t}}{J_1}=$"+"{0}".format(htip[i]))
#    plt.savefig("./" + foldername + results_foldername + "/FirstCorrelations_Comparison_"+addsave+".png")

In [ ]:
#######################################################################
#     Temperature crossing points (exp vs th) as a function of p      #
#######################################################################

In [ ]:
print(pswitch)
print(mexp)

In [ ]:
### 1 - m vs expm - p crossing

pmexpm = []

TMin_mexpm = []
TMax_mexpm = []
Diff2MinTest = []
Diff2MaxTest = []
plt.figure()
for i in range(n):
    
    Diff2Min = t_h_MeanM[i][:,0]-(abs(mexp)-mexperr)
    Diff2Max = t_h_MeanM[i][:,0]-(abs(mexp)+mexperr)

    #print(all(Diff2Min<0) or all(Diff2Max>0))
    Diff2MinTest.append(all(Diff2Min<0))
    Diff2MaxTest.append(all(Diff2Min>0))
    if not (all(Diff2Min<0) or all(Diff2Max>0)):
        print("Doing something")
        pmexpm.append(round(pswitch[i], 4))
        
        DiffmexpmMin = interp1d(temperatures_plots[i][:], 
                               t_h_MeanM[i][:,0]-(abs(mexp)-mexperr), kind = 'linear')
        DiffmexpmMax = interp1d(temperatures_plots[i][:], 
                               t_h_MeanM[i][:,0]-(abs(mexp)+mexperr), kind = 'linear')

        xnew = np.linspace(min(temperatures_plots[i][:]), max(temperatures_plots[i][:]), 1000)

           #middle:
        bracket = [0.1, 60]
        #min:
        plt.plot(xnew, DiffmexpmMin(xnew), '-', color = 'lightgreen')
        try:
            solution = root_scalar(DiffmexpmMin, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
            TMin_mexpm.append(round(solution.root, 6))
            plt.plot(solution.root, DiffmexpmMin(round(solution.root, 3)), '>', color = "red")
        except:
            TMin_mexpm.append(round(min(temperatures_plots[i][:]), 6))

        #max:
        plt.plot(xnew, DiffmexpmMax(xnew), '-', color = 'peachpuff')
        try:
            solution = root_scalar(DiffmexpmMax, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
            TMax_mexpm.append(round(solution.root, 6))
            plt.plot(solution.root, DiffmexpmMax(round(solution.root, 3)), '<', color = "red")
        except:
            TMax_mexpm.append(round(max(temperatures_plots[i][:]), 6))

In [ ]:
print("pmexpm = ", pmexpm )
print("TMin_mexpm= ",TMin_mexpm)
print("TMax_mexpm= ",TMax_mexpm)

In [ ]:
### 2 - c1 vs expc1 - p crossing

pc1expc1 = []

TMin_c1expc1 = []
TMax_c1expc1 = []
Diff2MinTest = []
Diff2MaxTest = []
plt.figure()
for i in range(n):
    
    Diff2Min = t_h_MeanFc[i][:,0,0]-(NN1exp-NN1experr)
    Diff2Max = t_h_MeanFc[i][:,0,0]-(NN1exp+NN1experr)

    #print(all(Diff2Min<0) or all(Diff2Max>0))
    Diff2MinTest.append(all(Diff2Min<0))
    Diff2MaxTest.append(all(Diff2Min>0))
    if not (all(Diff2Min<0) or all(Diff2Max>0)):
        print("Doing something")
        pc1expc1.append(round(pswitch[i], 4))
        
        Diffc1expc1Min = interp1d(temperatures_plots[i][:], 
                               t_h_MeanFc[i][:,0,0]-(NN1exp-NN1experr), kind = 'linear')
        Diffc1expc1Max = interp1d(temperatures_plots[i][:], 
                               t_h_MeanFc[i][:,0,0]-(NN1exp+NN1experr), kind = 'linear')

        xnew = np.linspace(min(temperatures_plots[i][:]), max(temperatures_plots[i][:]), 1000)

           #middle:
        bracket = [0.1, 60]
        #min:
        plt.plot(xnew, Diffc1expc1Min(xnew), '-', color = 'lightgreen')
        try:
            solution = root_scalar(Diffc1expc1Min, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
            TMin_c1expc1.append(round(solution.root, 6))
            plt.plot(solution.root, Diffc1expc1Min(round(solution.root, 3)), '>', color = "red")
        except:
            TMin_c1expc1.append(round(min(temperatures_plots[i][:]), 6))

        #max:
        plt.plot(xnew, Diffc1expc1Max(xnew), '-', color = 'peachpuff')
        try:
            solution = root_scalar(Diffc1expc1Max, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
            TMax_c1expc1.append(round(solution.root, 6))
            plt.plot(solution.root, Diffc1expc1Max(round(solution.root, 3)), '<', color = "red")
        except:
            TMax_c1expc1.append(round(max(temperatures_plots[i][:]), 6))

In [ ]:
print("pc1expc1 = ", pc1expc1 )
print("TMin_c1expc1= ",TMin_c1expc1)
print("TMax_c1expc1= ",TMax_c1expc1)

In [ ]:
### 3 - c2 vs expc2 - p crossing

pc2expc2 = []

TMin_c2expc2 = []
TMax_c2expc2 = []
Diff2MinTest = []
Diff2MaxTest = []
plt.figure()
for i in range(n):
    
    Diff2Min = t_h_MeanFc[i][:,0,1]-(NN2exp-NN2experr)
    Diff2Max = t_h_MeanFc[i][:,0,1]-(NN2exp+NN2experr)

    #print(all(Diff2Min<0) or all(Diff2Max>0))
    Diff2MinTest.append(all(Diff2Min<0))
    Diff2MaxTest.append(all(Diff2Min>0))
    if not (all(Diff2Min<0) or all(Diff2Max>0)):
        print("Doing something")
        pc2expc2.append(round(pswitch[i], 4))
        
        Diffc2expc2Max = interp1d(temperatures_plots[i][:], 
                               t_h_MeanFc[i][:,0,1]-(NN2exp-NN2experr), kind = 'linear')
        Diffc2expc2Min = interp1d(temperatures_plots[i][:], 
                               t_h_MeanFc[i][:,0,1]-(NN2exp+NN2experr), kind = 'linear')

        xnew = np.linspace(min(temperatures_plots[i][:]), max(temperatures_plots[i][:]), 1000)

           #middle:
        bracket = [0.1, 60]
        #min:
        plt.plot(xnew, Diffc2expc2Min(xnew), '-', color = 'lightgreen')
        try:
            solution = root_scalar(Diffc2expc2Min, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
            TMin_c2expc2.append(round(solution.root, 6))
            plt.plot(solution.root, Diffc2expc2Min(round(solution.root, 3)), '>', color = "red")
        except:
            TMin_c2expc2.append(round(min(temperatures_plots[i][:]), 6))

        #max:
        plt.plot(xnew, Diffc2expc2Max(xnew), '-', color = 'peachpuff')
        try:
            solution = root_scalar(Diffc2expc2Max, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
            TMax_c2expc2.append(round(solution.root, 6))
            plt.plot(solution.root, Diffc2expc2Max(round(solution.root, 3)), '<', color = "red")
        except:
            TMax_c2expc2.append(round(max(temperatures_plots[i][:]), 6))

In [ ]:
print("pc2expc2 = ", pc2expc2 )
print("TMin_c2expc2= ",TMin_c2expc2)
print("TMax_c2expc2= ",TMax_c2expc2)

In [ ]:
### 4 - c3par vs expc3par - p crossing

pc3parexpc3par = []

TMin_c3parexpc3par = []
TMax_c3parexpc3par = []
Diff2MinTest = []
Diff2MaxTest = []
plt.figure()
for i in range(n):
    
    Diff2Min = t_h_MeanFc[i][:,0,2]-(NN3pexp-NN3pexperr)
    Diff2Max = t_h_MeanFc[i][:,0,2]-(NN3pexp+NN3pexperr)

    #print(all(Diff2Min<0) or all(Diff2Max>0))
    Diff2MinTest.append(all(Diff2Min<0))
    Diff2MaxTest.append(all(Diff2Min>0))
    if not (all(Diff2Min<0) or all(Diff2Max>0)):
        print("Doing something")
        pc3parexpc3par.append(round(pswitch[i], 4))
        
        Diffc3parexpc3parMax = interp1d(temperatures_plots[i][:], 
                               t_h_MeanFc[i][:,0,2]-(NN3pexp-NN3pexperr), kind = 'linear')
        Diffc3parexpc3parMin = interp1d(temperatures_plots[i][:], 
                               t_h_MeanFc[i][:,0,2]-(NN3pexp+NN3pexperr), kind = 'linear')

        xnew = np.linspace(min(temperatures_plots[i][:]), max(temperatures_plots[i][:]), 1000)

           #middle:
        bracket = [0.1, 60]
        #min:
        plt.plot(xnew, Diffc3parexpc3parMin(xnew), '-', color = 'lightgreen')
        try:
            solution = root_scalar(Diffc3parexpc3parMin, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
            TMin_c3parexpc3par.append(round(solution.root, 6))
            plt.plot(solution.root, Diffc3parexpc3parMin(round(solution.root, 3)), '>', color = "red")
        except:
            TMin_c3parexpc3par.append(round(min(temperatures_plots[i][:]), 6))

        #max:
        plt.plot(xnew, Diffc3parexpc3parMax(xnew), '-', color = 'peachpuff')
        try:
            solution = root_scalar(Diffc3parexpc3parMax, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
            TMax_c3parexpc3par.append(round(solution.root, 6))
            plt.plot(solution.root, Diffc3parexpc3parMax(round(solution.root, 3)), '<', color = "red")
        except:
            TMax_c3parexpc3par.append(round(max(temperatures_plots[i][:]), 6))

In [ ]:
print("pc3parexpc3par = ", pc3parexpc3par )
print("TMin_c3parexpc3par= ",TMin_c3parexpc3par)
print("TMax_c3parexpc3par= ",TMax_c3parexpc3par)

In [ ]:
### 5 - c3star vs expc3star - p crossing

pc3starexpc3star = []

TMin_c3starexpc3star = []
TMax_c3starexpc3star = []
Diff2MinTest = []
Diff2MaxTest = []
plt.figure()
for i in range(n):
    
    Diff2Min = t_h_MeanFc[i][:,0,3]-(NN3sexp-NN3sexperr)
    Diff2Max = t_h_MeanFc[i][:,0,3]-(NN3sexp+NN3sexperr)

    
    Diff2MinTest.append(all(Diff2Min<0))
    Diff2MaxTest.append(all(Diff2Min>0))
    if not (all(Diff2Min<0) or all(Diff2Max>0)):
        print("Doing something")
        pc3starexpc3star.append(round(pswitch[i], 4))
        
        Diffc3starexpc3starMin = interp1d(temperatures_plots[i][:], 
                               t_h_MeanFc[i][:,0,3]-(NN3sexp-NN3sexperr), kind = 'linear')
        Diffc3starexpc3starMax = interp1d(temperatures_plots[i][:], 
                               t_h_MeanFc[i][:,0,3]-(NN3sexp+NN3sexperr), kind = 'linear')

        xnew = np.linspace(min(temperatures_plots[i][:]), max(temperatures_plots[i][:]), 1000)

           #middle:
        bracket = [0.1, 55]
        #min:
        plt.plot(xnew, Diffc3starexpc3starMin(xnew), '-', color = 'lightgreen')
        try:
            solution = root_scalar(Diffc3starexpc3starMin, bracket = bracket, xtol = 1e-5)#, method = 'bisect')
            TMin_c3starexpc3star.append(round(solution.root, 6))
            plt.plot(solution.root, Diffc3starexpc3starMin(round(solution.root, 3)), '>', color = "green")
        except:
            TMin_c3starexpc3star.append(round(min(temperatures_plots[i][:]), 6))

        #max:
        plt.plot(xnew, Diffc3starexpc3starMax(xnew), '-', color = 'peachpuff')
        try:
            solution = root_scalar(Diffc3starexpc3starMax, bracket = bracket, xtol = 1e-3)#, method = 'bisect')
            TMax_c3starexpc3star.append(round(solution.root, 6))
            plt.plot(solution.root, Diffc3starexpc3starMax(round(solution.root, 3)), '<', color = "red")
        except:
            TMax_c3starexpc3star.append(round(max(temperatures_plots[i][:]), 6))
    else:
        print(max(Diff2Min), min(Diff2Max))
        pc3starexpc3star.append(round(pswitch[i], 4))
        TMin_c3starexpc3star.append(round(max(temperatures_plots[i][:]), 6))
        TMax_c3starexpc3star.append(round(max(temperatures_plots[i][:]), 6))
        
plt.ylim([-0.01,0.01])
plt.plot([0,60],[0,0])

In [ ]:
print("pc3starexpc3star = ", pc3starexpc3star )
print("TMin_c3starexpc3star= ",TMin_c3starexpc3star)
print("TMax_c3starexpc3star= ",TMax_c3starexpc3star)

In [ ]:
### 6 - c vs c3par - p crossing

pc2c3par = []

Tcross_c2c3par = []
plt.figure()
for i in range(n):
    
    Diff = t_h_MeanFc[i][:,0,1]-t_h_MeanFc[i][:,0,2]
    
    if not (all(Diff<0) or all(Diff>0)):
        print("Doing something")
        print(Diff<0)
        pc2c3par.append(round(pswitch[i], 4))
        
        Diffc2c3par = interp1d(temperatures_plots[i][:], 
                               t_h_MeanFc[i][:,0,1]- t_h_MeanFc[i][:,0,2],
                               kind = 'linear')
        xnew = np.linspace(min(temperatures_plots[i][:]),
                           max(temperatures_plots[i][:]), 1000)

           #middle:
        bracket = [0.1, 20]
        #min:
        plt.plot(xnew, Diffc2c3par(xnew), '-', color = 'lightgreen')
        try:
            solution = root_scalar(Diffc2c3par, bracket = bracket, xtol = 1e-3)#, method = 'bisect')
            Tcross_c2c3par.append(round(solution.root, 6))
            plt.plot(solution.root, Diffc2c3par(round(solution.root, 3)), '>', color = "red")
        except:
            Tcross_c2c3par.append(round(min(temperatures_plots[i][:]), 6))

    else:
        print(max(Diff), min(Diff))
        
plt.plot([0.1, 60], [0,0])
plt.ylim([-0.01, 0.01])

In [ ]:
print("pc2c3par = ", pc2c3par )
print("Tcross_c2c3par= ",Tcross_c2c3par)

In [ ]:
######################################################################
#                   TEMPERATURE SCAN VERSION                          #
######################################################################

In [ ]:
Tmin = 0.1
Tmax = 62
idTmin = []
idTmax = []

for i in range(n):
    idTmin.append(np.abs(np.array(temperatures_plots[i])-Tmin).argmin())
    idTmax.append(np.abs(np.array(temperatures_plots[i])-Tmax).argmin())

print(idTmin)
print(idTmax)

In [ ]:
temperatures_plots[i].shape

In [ ]:
t_h_MeanM[i].shape

In [ ]:
mT = []
ErrmT = []
NN1 = []
ErrNN1 = []
NN2 = []
ErrNN2 = []
NN3par = []
ErrNN3par = []
NN3star = []
ErrNN3star = []
for i in range(n):
    #for idT in range(idTmin, idTmax):
    mT.append(t_h_MeanM[i][idTmin[i]:idTmax[i]+1,0])
    ErrmT.append(np.sqrt(t_h_varMeanM[i][idTmin[i]:idTmax[i]+1,0]))
    NN1.append(t_h_MeanFc[i][idTmin[i]:idTmax[i]+1,0,0])
    ErrNN1.append(np.sqrt(t_h_varMeanFc[i][idTmin[i]:idTmax[i]+1,0,0]))
    NN2.append(t_h_MeanFc[i][idTmin[i]:idTmax[i]+1,0,1])
    ErrNN2.append(np.sqrt(t_h_varMeanFc[i][idTmin[i]:idTmax[i]+1,0,1]))
    NN3par.append(t_h_MeanFc[i][idTmin[i]:idTmax[i]+1,0,2])
    ErrNN3par.append(np.sqrt(t_h_varMeanFc[i][idTmin[i]:idTmax[i]+1,0,2]))
    NN3star.append(t_h_MeanFc[i][idTmin[i]:idTmax[i]+1,0,3])
    ErrNN3star.append(np.sqrt(t_h_varMeanFc[i][idTmin[i]:idTmax[i]+1,0,3]))


    
mT = np.array(mT).transpose()
ErrmT = np.array(ErrmT).transpose()

NN1 = np.array(NN1).transpose()
ErrNN1 = np.array(ErrNN1).transpose()

NN2 = np.array(NN2).transpose()
ErrNN2 = np.array(ErrNN2).transpose()

NN3par = np.array(NN3par).transpose()
ErrNN3par = np.array(ErrNN3par).transpose()

NN3star = np.array(NN3star).transpose()
ErrNN3star = np.array(ErrNN3star).transpose()

In [ ]:
mT.shape

In [ ]:
temperatures_plots[0]

In [ ]:
### 1 - m vs expm - p crossing

pMin_mexpm = []
pMax_mexpm = []
T_mexpm = []

plt.figure()

Diff2MinTest = []
Diff2MaxTest = []
for idT in range(idTmax[0]-idTmin[0]+1):
    print(idT)
    Diff2Min = mT[idT+idTmin[0],:]-(abs(mexp)-mexperr)
    Diff2Max = ErrmT[idT+idTmin[0],:]-(abs(mexp)+mexperr)
    
    
    Diff2MinTest.append(all(Diff2Min<0))
    Diff2MaxTest.append(all(Diff2Min>0))
    if not (all(Diff2Min<0) or all(Diff2Max>0)):
        print(temperatures_plots[0][idT+idTmin[0]])
        T_mexpm.append(round(temperatures_plots[0][idT+idTmin[0]], 6))
        
        DiffmexpmMin = interp1d(pswitch, 
                               mT[idT,:]-(abs(mexp)-mexperr), kind = 'linear')
        DiffmexpmMax = interp1d(pswitch, 
                               mT[idT,:]-(abs(mexp)+mexperr), kind = 'linear')

        xnew = np.linspace(min(pswitch), max(pswitch), 1000)

           #middle:
        bracket = [min(pswitch), max(pswitch)]
        #min:
        plt.plot(xnew, DiffmexpmMin(xnew), '-', color = 'lightgreen', alpha = 0.2)
        try:
            solution = root_scalar(DiffmexpmMin, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
            pMin_mexpm.append(round(solution.root, 6))
            plt.plot(solution.root, DiffmexpmMin(round(solution.root, 3)), '>', color = "green")
        except:
            pMin_mexpm.append(round(min(pswitch), 6))

        #max:
        plt.plot(xnew, DiffmexpmMax(xnew), '-', color = 'peachpuff', alpha = 0.2)
        try:
            solution = root_scalar(DiffmexpmMax, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
            pMax_mexpm.append(round(solution.root, 6))
            plt.plot(solution.root, DiffmexpmMax(round(solution.root, 3)), '<', color = "red")
        except:
            pMax_mexpm.append(round(max(pswitch), 6))
    else:
        print(temperatures_plots[0][idT+idTmin[0]])

In [ ]:
print("pMin_mexpm = ", pMin_mexpm)
print("pMax_mexpm = ", pMax_mexpm)
print("T_mexpm = ", T_mexpm)

In [ ]:
### 2 - c1 vs expc1 - p crossing

pMin_c1expc1 = []
pMax_c1expc1 = []
T_c1expc1 = []

plt.figure()

Diff2MinTest = []
Diff2MaxTest = []
for idT in range(idTmax[0]-idTmin[0]+1):
    #print(idT)
    Diff2Min = NN1[idT+idTmin[0],:]-(NN1exp-NN1experr)
    Diff2Max = NN1[idT+idTmin[0],:]-(NN1exp+NN1experr)
    
    
    Diff2MinTest.append(all(Diff2Min<0))
    Diff2MaxTest.append(all(Diff2Min>0))
    if not (all(Diff2Min<0) or all(Diff2Max>0)):
        print(temperatures_plots[0][idT+idTmin[0]])
        T_c1expc1.append(round(temperatures_plots[0][idT+idTmin[0]], 6))
        
        Diffc1expc1Min = interp1d(pswitch, 
                               NN1[idT,:]-(NN1exp-NN1experr), kind = 'linear')
        Diffc1expc1Max = interp1d(pswitch, 
                               NN1[idT,:]-(NN1exp+NN1experr), kind = 'linear')

        xnew = np.linspace(min(pswitch), max(pswitch), 1000)

           #middle:
        bracket = [min(pswitch), max(pswitch)]
        #min:
        plt.plot(xnew, Diffc1expc1Min(xnew), '-', color = 'peachpuff', alpha = 0.2)
        try:
            solution = root_scalar(Diffc1expc1Min, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
            pMax_c1expc1.append(round(solution.root, 6))
            plt.plot(solution.root, Diffc1expc1Min(round(solution.root, 3)), '<', color = "red")
        except:
            pMax_c1expc1.append(round(max(pswitch), 6))

        #max:
        plt.plot(xnew, Diffc1expc1Max(xnew), '-', color = 'green', alpha = 0.2)
        try:
            solution = root_scalar(Diffc1expc1Max, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
            pMin_c1expc1.append(round(solution.root, 6))
            plt.plot(solution.root, Diffc1expc1Max(round(solution.root, 3)), '>', color = "lightgreen")
        except:
            pMin_c1expc1.append(round(min(pswitch), 6))

In [ ]:
print("pMin_c1expc1 = ", pMin_c1expc1)
print("pMax_c1expc1 = ", pMax_c1expc1)
print("T_c1expc1 = ", T_c1expc1)

In [ ]:
###3 - c2 vs expc2 - p crossing

pMin_c2expc2 = []
pMax_c2expc2 = []
T_c2expc2 = []

plt.figure()

Diff2MinTest = []
Diff2MaxTest = []
for idT in range(idTmax[0]-idTmin[0]+1):
    #print(idT)
    Diff2Min = NN2[idT+idTmin[0],:]-(NN2exp-NN2experr)
    Diff2Max = NN2[idT+idTmin[0],:]-(NN2exp+NN2experr)
    
    
    Diff2MinTest.append(all(Diff2Min<0))
    Diff2MaxTest.append(all(Diff2Min>0))
    if not (all(Diff2Min<0) or all(Diff2Max>0)):
        print(temperatures_plots[0][idT+idTmin[0]])
        T_c2expc2.append(round(temperatures_plots[0][idT+idTmin[0]], 6))
        
        Diffc2expc2Min = interp1d(pswitch, 
                               NN2[idT,:]-(NN2exp-NN2experr), kind = 'linear')
        Diffc2expc2Max = interp1d(pswitch, 
                               NN2[idT,:]-(NN2exp+NN2experr), kind = 'linear')

        xnew = np.linspace(min(pswitch), max(pswitch), 1000)

           #middle:
        bracket = [min(pswitch), max(pswitch)]
        #min:
        plt.plot(xnew, Diffc2expc2Min(xnew), '-', color = 'lightgreen', alpha = 0.2)
        try:
            solution = root_scalar(Diffc2expc2Min, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
            pMin_c2expc2.append(round(solution.root, 6))
            plt.plot(solution.root, Diffc2expc2Min(round(solution.root, 3)), '>', color = "green")
        except:
            pMin_c2expc2.append(round(min(pswitch), 6))

        #max:
        plt.plot(xnew, Diffc2expc2Max(xnew), '-', color = 'peachpuff', alpha = 0.2)
        try:
            solution = root_scalar(Diffc2expc2Max, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
            pMax_c2expc2.append(round(solution.root, 6))
            plt.plot(solution.root, Diffc2expc2Max(round(solution.root, 3)), '<', color = "red")
        except:
            pMax_c2expc2.append(round(max(pswitch), 6))

In [ ]:
print("pMin_c2expc2 = ", pMin_c2expc2)
print("pMax_c2expc2 = ", pMax_c2expc2)
print("T_c2expc2 = ", T_c2expc2)

In [ ]:
###4 - c3p vs expc3p - p crossing

pMin_c3pexpc3p = []
pMax_c3pexpc3p = []
T_c3pexpc3p = []

plt.figure()

Diff2MinTest = []
Diff2MaxTest = []
for idT in range(idTmax[0]-idTmin[0]+1):
    #print(idT)
    Diff2Min = NN3par[idT+idTmin[0],:]-(NN3pexp-NN3pexperr)
    Diff2Max = NN3par[idT+idTmin[0],:]-(NN3pexp+NN3pexperr)
    
    
    Diff2MinTest.append(all(Diff2Min<0))
    Diff2MaxTest.append(all(Diff2Min>0))
    if not (all(Diff2Min<0) or all(Diff2Max>0)):
        print(temperatures_plots[0][idT+idTmin[0]])
        T_c3pexpc3p.append(round(temperatures_plots[0][idT+idTmin[0]], 6))
        
        Diffc3pexpc3pMin = interp1d(pswitch, 
                               NN3par[idT,:]-(NN3pexp-NN3pexperr), kind = 'linear')
        Diffc3pexpc3pMax = interp1d(pswitch, 
                               NN3par[idT,:]-(NN3pexp+NN3pexperr), kind = 'linear')

        xnew = np.linspace(min(pswitch), max(pswitch), 1000)

           #middle:
        bracket = [min(pswitch), 1]
        #min:
        plt.plot(xnew, Diffc3pexpc3pMin(xnew), '-', color = 'lightgreen', alpha = 0.4)
        try:
            solution = root_scalar(Diffc3pexpc3pMin, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
            pMin_c3pexpc3p.append(round(solution.root, 6))
            plt.plot(solution.root, Diffc3pexpc3pMin(round(solution.root, 3)), '>', color = "green")
        except:
            pMin_c3pexpc3p.append(round(min(pswitch), 6))

        #max:
        plt.plot(xnew, Diffc3pexpc3pMax(xnew), '-', color = 'peachpuff', alpha = 0.2)
        try:
            solution = root_scalar(Diffc3pexpc3pMax, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
            pMax_c3pexpc3p.append(round(solution.root, 6))
            plt.plot(solution.root, Diffc3pexpc3pMax(round(solution.root, 3)), '<', color = "red")
        except:
            pMax_c3pexpc3p.append(round(max(pswitch), 6))
            
plt.plot([0.05, 1], [0,0])
plt.ylim([-0.01, None])

In [ ]:
print("pMin_c3pexpc3p = ", pMin_c3pexpc3p)
print("pMax_c3pexpc3p = ", pMax_c3pexpc3p)
print("T_c3pexpc3p = ", T_c3pexpc3p)

In [ ]:
###5 - c3s vs expc3s - p crossing

pMin_c3sexpc3s = []
pMax_c3sexpc3s = []
T_c3sexpc3s = []

plt.figure()

Diff2MinTest = []
Diff2MaxTest = []
for idT in range(idTmax[0]-idTmin[0]+1):
    #print(idT)
    Diff2Min = NN3star[idT+idTmin[0],:]-(NN3sexp-NN3sexperr)
    Diff2Max = NN3star[idT+idTmin[0],:]-(NN3sexp+NN3sexperr)
    
    
    Diff2MinTest.append(all(Diff2Min<0))
    Diff2MaxTest.append(all(Diff2Min>0))
    if not (all(Diff2Min<0) or all(Diff2Max>0)):
        print(temperatures_plots[0][idT+idTmin[0]])
        T_c3sexpc3s.append(round(temperatures_plots[0][idT+idTmin[0]], 6))
        
        Diffc3sexpc3sMin = interp1d(pswitch, 
                               NN3star[idT,:]-(NN3sexp-NN3sexperr+1e-5), kind = 'linear')
        Diffc3sexpc3sMax = interp1d(pswitch, 
                               NN3star[idT,:]-(NN3sexp+NN3sexperr+1e-5), kind = 'linear')

        xnew = np.linspace(min(pswitch), max(pswitch), 1000)

           #middle:
        bracket = [min(pswitch), max(pswitch)]
        #min:
        plt.plot(xnew, Diffc3sexpc3sMin(xnew), '-', color = 'peachpuff', alpha = 0.5)
        try:
            solution = root_scalar(Diffc3sexpc3sMin, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
            pMax_c3sexpc3s.append(round(solution.root, 6))
            plt.plot(solution.root, Diffc3sexpc3sMin(round(solution.root, 3)), '<', color = "red")
        except:
            pMax_c3sexpc3s.append(round(max(pswitch), 6))

        #max:
        plt.plot(xnew, Diffc3sexpc3sMax(xnew), '-', color = 'lightgreen', alpha = 0.2)
        try:
            solution = root_scalar(Diffc3sexpc3sMax, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
            pMin_c3sexpc3s.append(round(solution.root, 6))
            plt.plot(solution.root, Diffc3sexpc3sMax(round(solution.root, 3)), '>', color = "green")
        except:
            pMin_c3sexpc3s.append(round(min(pswitch), 6))
            
plt.ylim([-0.005,None])

In [ ]:
print("pMin_c3sexpc3s = ", pMin_c3sexpc3s)
print("pMax_c3sexpc3s = ", pMax_c3sexpc3s)
print("T_c3sexpc3s = ", T_c3sexpc3s)

In [ ]:
###5 - c3s vs expc3s - p crossing

HP_pMin_c3sexpc3s = []
HP_pMax_c3sexpc3s = []
HP_T_c3sexpc3s = []

plt.figure()

Diff2MinTest = []
Diff2MaxTest = []
for idT in range(idTmax[0]-idTmin[0]+1):
    #print(idT)
    Diff2Min = NN3star[idT+idTmin[0],:]-(NN3sexp-NN3sexperr)
    Diff2Max = NN3star[idT+idTmin[0],:]-(NN3sexp+NN3sexperr)
    
    
    Diff2MinTest.append(all(Diff2Min<0))
    Diff2MaxTest.append(all(Diff2Min>0))
    if not (all(Diff2Min<0) or all(Diff2Max>0)):
        print(temperatures_plots[0][idT+idTmin[0]])
        HP_T_c3sexpc3s.append(round(temperatures_plots[0][idT+idTmin[0]], 6))
        
        Diffc3sexpc3sMin = interp1d(pswitch, 
                               NN3star[idT,:]-(NN3sexp-NN3sexperr+1e-5), kind = 'linear')
        Diffc3sexpc3sMax = interp1d(pswitch, 
                               NN3star[idT,:]-(NN3sexp+NN3sexperr+1e-5), kind = 'linear')

        xnew = np.linspace(0.6, max(pswitch), 1000)

           #middle:
        bracket = [0.6, max(pswitch)]
        #min:
        plt.plot(xnew, Diffc3sexpc3sMin(xnew), '-', color = 'lightgreen', alpha = 0.5)
        try:
            solution = root_scalar(Diffc3sexpc3sMin, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
            HP_pMin_c3sexpc3s.append(round(solution.root, 6))
            plt.plot(solution.root, Diffc3sexpc3sMin(round(solution.root, 3)), '>', color = "green")
        except:
            HP_pMin_c3sexpc3s.append(round(max(pswitch), 6))

        #max:
        plt.plot(xnew, Diffc3sexpc3sMax(xnew), '-', color = 'peachpuff', alpha = 0.2)
        try:
            solution = root_scalar(Diffc3sexpc3sMax, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
            HP_pMax_c3sexpc3s.append(round(solution.root, 6))
            plt.plot(solution.root, Diffc3sexpc3sMax(round(solution.root, 3)), '<', color = "red")
        except:
            HP_pMax_c3sexpc3s.append(round(max(pswitch), 6))
            
plt.ylim([-0.005,None])

In [ ]:
print("HP_pMin_c3sexpc3s = ", HP_pMin_c3sexpc3s)
print("HP_pMax_c3sexpc3s = ", HP_pMax_c3sexpc3s)
print("HP_T_c3sexpc3s = ", HP_T_c3sexpc3s)

In [ ]:
### 7 - c2 vs c3|| - pswitch crossing

crossing_c2c3par = []
Tcrossing = []
Tfailedcrossing_c2c3par = []
crossing_c2c3parMin = []
TMinCrossing = []
Tfailedcrossing_c2c3parMin = []
crossing_c2c3parMax = []
TMaxCrossing = []
Tfailedcrossing_c2c3parMax = []

for idT in range(idTmax[0]-idTmin[0]+1):
    Diffc2c3par = interp1d(pswitch,  NN2[idT,:]-NN3par[idT,:], kind = 'linear')
    Diffc2c3parMin = interp1d(pswitch, 
                            NN2[idT,:]-NN3par[idT,:]-np.sqrt(ErrNN2[idT,:]+ErrNN3par[idT,:]), kind = 'linear')
    Diffc2c3parMax = interp1d(pswitch, 
                            NN2[idT,:]-NN3par[idT,:]+np.sqrt(ErrNN2[idT,:]+ErrNN3par[idT,:]), kind = 'linear')
   
    xnew = np.linspace(min(pswitch), max(pswitch), 1000)
    
    #plt.figure()
    #plt.plot([min(pswitch), max(pswitch)], [0, 0], color = 'orange')
    #plt.plot(pswitch,  NN2[idT,:]-NN3par[idT,:], 'k.',
    #                 xnew, Diffc2c3par(xnew), '-')
    #
    #plt.fill_between(xnew, Diffc2c3parMin(xnew),Diffc2c3parMax(xnew), alpha = 0.2, color = 'lightblue')
    
    #middle:
    bracket = [0.05, 1]
    countfail = 0
    plt.plot(xnew, Diffc2c3par(xnew), '-', color = 'lightblue')
    try:
        solution = root_scalar(Diffc2c3par, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
        crossing_c2c3par.append(round(solution.root, 6))
        plt.plot(solution.root, Diffc2c3par(round(solution.root, 3)), 'o', color = "red")
        Tcrossing.append(round(temperatures_plots[0][idT], 4))
    except:
        Tfailedcrossing_c2c3par.append(temperatures_plots[0][idT])
        countfail +=1
    #min:
    #plt.plot(xnew, Diffc2c3parMin(xnew), '-', color = 'lightgreen')
    #try:
    #    solution = root_scalar(Diffc2c3parMin, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
    #    crossing_c2c3parMin.append(round(solution.root, 6))
    #    TMinCrossing.append(round(temperatures_plots[0][idT], 4))
    #    plt.plot(solution.root, Diffc2c3parMin(round(solution.root, 3)), '>', color = "red")
    #except:
    #    Tfailedcrossing_c2c3parMin.append(temperatures_plots[0][idT])
    #    countfail +=1
    #    
    ##max:
    #plt.plot(xnew, Diffc2c3parMax(xnew), '-', color = 'peachpuff')
    #try:
    #    solution = root_scalar(Diffc2c3parMax, bracket = bracket, xtol = 1e-4)#, method = 'bisect')
    #    crossing_c2c3parMax.append(round(solution.root, 6))
    #    TMaxCrossing.append(round(temperatures_plots[0][idT], 4))
    #    plt.plot(solution.root, Diffc2c3parMax(round(solution.root, 3)), '<', color = "red")
    #except:
    #    Tfailedcrossing_c2c3parMax.append(temperatures_plots[0][idT])
    #    countfail +=1
    
plt.plot([0.05, 1], [0,0])
plt.ylim([-0.005, 0.02])

In [ ]:
print("T_c2c3par = ", Tcrossing)
print("p_c2c3par = ", crossing_c2c3par)
print("T_c2c3parMin = ", TMinCrossing)
print("p_c2c3parMin = ", crossing_c2c3parMin)
print("T_c2c3parMax = ", TMaxCrossing)
print("p_c2c3parMax = ", crossing_c2c3parMax)